In [5]:
import urllib.request
import os
import zipfile
import csv
import pygal
import lxml
import pygal_maps_world

In [2]:
# lista para receber o arquivo numero_identificador_lattes_zzzz.csv
identificador_list_cvs = list()
# dicionario para receber o arquivo tab_area_conhecimento_zzzz.csv
area_dict_csv = {}
# dicionario para receber o arquivo tab_nivel_formacao_zzzz.csv
nivel_formacao_dict_csv = {}

In [3]:
# url dos arquivos da plataforma lattes, disponivel em:  http://memoria.cnpq.br/web/portal-lattes/extracoes-de-dados
url = "http://memoria.cnpq.br/documents/313759/6935115b-a5e8-4cfb-96cf-00e2fd11c96c"
print("URL lattes: " + url)

URL lattes: http://memoria.cnpq.br/documents/313759/6935115b-a5e8-4cfb-96cf-00e2fd11c96c


In [7]:
# definicao do arquivo destino na maquina do usuario
arquivo_lattes = os.path.expanduser("~/lattes.zip")
print("Arquivo de download destino: " + arquivo_lattes)

Arquivo de download destino: /Users/eduardofelipe/lattes.zip


In [8]:
dir_destino = os.path.dirname(arquivo_lattes)
print("Diretório de trabalho local: " + dir_destino)

Diretório de trabalho local: /Users/eduardofelipe


In [9]:
print("Iniciando download do arquivo na plataforma lattes. Processo demorado...")
urllib.request.urlretrieve(url, arquivo_lattes)
print("Download concluído: " + arquivo_lattes)

Iniciando download do arquivo na plataforma lattes. Processo demorado...
Download concluído: /Users/eduardofelipe/lattes.zip


In [10]:
print("Descompactando o arquivo ZIP... ")
with zipfile.ZipFile(arquivo_lattes, "r") as zip_ref:
    zip_ref.extractall(dir_destino)
print("Arquivos da plataforma lattes extraídos em: " + dir_destino)

Descompactando o arquivo ZIP... 
Arquivos da plataforma lattes extraídos em: /Users/eduardofelipe


In [11]:
def importa_csv_para_lista(arq):
    print("Importando o arquivo: " + arq + " para estrutura de dados em memória a fim de permitir análise.")
    with open(arq, 'r', encoding='latin-1') as f:
        reader = csv.reader(f, delimiter=';')
        lista = list(reader)
    print("Importação do arquivo: " + arq + " para a memória concluída.")
    return lista

In [12]:
def importa_csv_para_dicionario(arq):
    print("Importando o arquivo: " + arq + " para estrutura de dados em memória a fim de permitir análise.")
    dicionario = {}
    reader = csv.reader(open(arq, 'r', encoding='latin-1'))
    for row in reader:
        aux_linha = row[0]
        aux_linha_str = aux_linha.split(";")
        chave = aux_linha_str[0]
        valor = aux_linha_str[1]
        dicionario[chave] = valor
    print("Importação do arquivo: " + arq + " para a memória concluída.")
    return dicionario

In [13]:
for file in os.listdir(dir_destino):
    if file.endswith(".csv"):
        if file.startswith("numero_identificador_lattes"):
            identificador_arq = os.path.join(dir_destino, file)
            identificador_list_cvs = importa_csv_para_lista(identificador_arq)

        if file.startswith("tab_area_conhecimento"):
            area_arq = os.path.join(dir_destino, file)
            area_dict_csv = importa_csv_para_dicionario(area_arq)

        if file.startswith("tab_nivel_formacao"):
            nivel_arq = os.path.join(dir_destino, file)
            nivel_formacao_dict_csv = importa_csv_para_dicionario(nivel_arq)


Importando o arquivo: /Users/eduardofelipe/numero_identificador_lattes_20170708.csv para estrutura de dados em memória a fim de permitir análise.
Importação do arquivo: /Users/eduardofelipe/numero_identificador_lattes_20170708.csv para a memória concluída.
Importando o arquivo: /Users/eduardofelipe/tab_nivel_formacao_20170708.csv para estrutura de dados em memória a fim de permitir análise.
Importação do arquivo: /Users/eduardofelipe/tab_nivel_formacao_20170708.csv para a memória concluída.
Importando o arquivo: /Users/eduardofelipe/tab_area_conhecimento_20170708.csv para estrutura de dados em memória a fim de permitir análise.
Importação do arquivo: /Users/eduardofelipe/tab_area_conhecimento_20170708.csv para a memória concluída.


In [14]:
# o algoritmo deve varrer a lista do identificador que eh o arquivo principal
# analisar cada linha do arquivo e trabalhar com geraçao estatistica
# em estruturas de dados em python, denominadas dicionario

nacionalidade_dict = {}
area_conhecimento_dict = {}
nivel_formacao_dict = {}

i = 1
valor = 0
while i < len(identificador_list_cvs):
    # le a linha
    aux_linha = identificador_list_cvs[i]
    # e extrai as strings para cada estatistica
    aux_nacionalidade = aux_linha[1]
    aux_area = aux_linha[3]
    aux_nivel = aux_linha[4]

    # trata a estatistica da nacionalidade
    valor_nacionalidade = nacionalidade_dict.get(aux_nacionalidade)
    if valor_nacionalidade is None:
        nacionalidade_dict[aux_nacionalidade] = 1
    else:
        valor_nacionalidade = valor_nacionalidade + 1
        nacionalidade_dict[aux_nacionalidade] = valor_nacionalidade

    # trata a estatistica da area de conhecimento
    valor_area = area_conhecimento_dict.get(aux_area)
    if valor_area is None:
        area_conhecimento_dict[aux_area] = 1
    else:
        valor_area = valor_area + 1
        area_conhecimento_dict[aux_area] = valor_area

    # trata a estatistica do nivel de formacao
    valor_nivel = nivel_formacao_dict.get(aux_nivel)
    if valor_nivel is None:
        nivel_formacao_dict[aux_nivel] = 1
    else:
        valor_nivel = valor_nivel + 1
        nivel_formacao_dict[aux_nivel] = valor_nivel

    i = i + 1


In [15]:
# tratar os dicionarios para estruturas de listas
# a fim de permitir ordenacao e gerar graficos de melhor leitura
nacionalidade_list = list()
nacionalidade_list = sorted(nacionalidade_dict.items(), key = lambda x: x[1], reverse=True )

area_list = list()
area_list = sorted(area_conhecimento_dict.items(), key = lambda x: x[1], reverse=True)

nivel_formacao_list = list()
nivel_formacao_list = sorted(nivel_formacao_dict.items(), key = lambda x: x[1], reverse=True)


In [16]:
# ######################################################
# GRAFICOS
# gerar o grafico de barras de pesquisadores estrageiros
nacionalidade_chart = pygal.HorizontalBar()
nacionalidade_chart.title = 'Pesquisadores estrangeiros cadastrados na Plataforma Lattes'
i = 0
while i < len(nacionalidade_list):
    aux_linha = nacionalidade_list[i]
    aux_nacionalidade = aux_linha[0]
    valor_nacionalidade = aux_linha[1]
    if aux_nacionalidade != "BRA":
        nacionalidade_chart.add(aux_nacionalidade, valor_nacionalidade)
    i = i + 1
nacionalidade_chart.render_in_browser()


file:///var/folders/s_/lcnjnb_n1zx8nsb881vyh26r0000gn/T/tmp28x52r8k.html


In [17]:
# gerar o grafico/mapa de pesquisadores por paises (incluindo Brasil desta vez)
worldmap_chart = pygal.maps.world.World()
worldmap_chart.title = 'Pesquisadores cadastrados na plataforma Lattes - por países.'
worldmap_chart.add('Mês/Ano', {
  'af': 14,
  'bd': 1,
  'by': 3,
  'cn': 1000,
  'gm': 9,
  'in': 1,
  'ir': 314,
  'iq': 129,
  'jp': 7
})
worldmap_chart.render_in_browser()

file:///var/folders/s_/lcnjnb_n1zx8nsb881vyh26r0000gn/T/tmplubfwdf4.html


In [18]:
# gerar o grafico de area de atuaçao
area_chart = pygal.HorizontalBar()
area_chart.title = "Área de atuação dos pesquisadores"
i = 0
while i < len(area_list):
    aux_linha = area_list[i]
    aux_area = aux_linha[0]
    aux_area_extenso = area_dict_csv.get(aux_area)
    valor_area = aux_linha[1]
    if bool(aux_area_extenso and aux_area_extenso.strip()):
        area_chart.add(aux_area_extenso, valor_area)
    i = i + 1
area_chart.render_in_browser()

file:///var/folders/s_/lcnjnb_n1zx8nsb881vyh26r0000gn/T/tmpj9bn_3db.html


In [19]:
# gerar o grafico de nivel de formacao
nivel_formacao_chart = pygal.HorizontalBar()
nivel_formacao_chart.title = "Nível de formação dos pesquisadores"
i = 0
while i < len(nivel_formacao_list):
    aux_linha = nivel_formacao_list[i]
    aux_nivel = aux_linha[0]
    if (bool(aux_nivel and aux_nivel.strip())):
        aux_nivel_extenso = nivel_formacao_dict_csv.get(aux_nivel)
    else:
        aux_nivel_extenso = "Não informado"
    valor_nivel = aux_linha[1]
    nivel_formacao_chart.add(aux_nivel_extenso, valor_nivel)
    i = i + 1
nivel_formacao_chart.render_in_browser()

file:///var/folders/s_/lcnjnb_n1zx8nsb881vyh26r0000gn/T/tmpoeccqshv.html


In [21]:

# excluir os arquivos de download no final do processo
print("Excluindo o arquivo: " + arquivo_lattes + " após a descompactação.")
#os.remove(file_name)
print("Excluindo o arquivo: " + identificador_arq)
#os.remove(arquivo_identificador)
print("Excluindo o arquivo: " + area_arq)
#os.remove(arquivo_area)
print("Excluindo o arquivo: " + nivel_arq)
#os.remove(arquivo_nivel)


Excluindo o arquivo: /Users/eduardofelipe/lattes.zip após a descompactação.
Excluindo o arquivo: /Users/eduardofelipe/numero_identificador_lattes_20170708.csv
Excluindo o arquivo: /Users/eduardofelipe/tab_area_conhecimento_20170708.csv
Excluindo o arquivo: /Users/eduardofelipe/tab_nivel_formacao_20170708.csv
